In [1]:
from corpus_characterizer import generator_chunker
import numpy as np
import os
# from sklearn.metrics import mean_squared_error,mean_absolute_error, median_absolute_error, mean_squared_log_error, explained_variance_score, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, explained_variance_score, \
    r2_score
import pandas as pd
from scipy.stats import describe, kurtosistest, skewtest, normaltest
from Conv1D_LSTM_Ensemble import pair_generator_1dconv_lstm_bagged
from Conv1D_ActivationSearch_BigLoop import pair_generator_1dconv_lstm #NOT BAGGED
from AuxRegressor import create_testing_set, create_training_set
import matplotlib.pyplot as plt 

# @@@@@@@@@@@@@@ RELATIVE PATHS @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Base_Path = "./"
image_path = "./images/"
train_path = "./train/"
test_path = "./test/"
analysis_path = "./analysis/"
chunker_path = analysis_path + "chunker/"
preds_path = analysis_path + "preds/"

#create the data-pair filenames (using zip), use the helper methods
train_set_filenames = create_training_set()
test_set_filenames = create_testing_set()
print(train_set_filenames)
print(test_set_filenames)

Using Theano backend.


after shuffling: [('sequence_2b_182_3_fv1b.npy', 'sequence_2b_182_3_label_fv1b.npy'), ('sequence_2b_222_3_fv1b.npy', 'sequence_2b_222_3_label_fv1b.npy'), ('sequence_1b_71_2_fv1b.npy', 'sequence_1b_71_2_label_fv1b.npy'), ('sequence_1b_102_3_fv1b.npy', 'sequence_1b_102_3_label_fv1b.npy'), ('sequence_1c_27_8_fv1b.npy', 'sequence_1c_27_8_label_fv1b.npy'), ('sequence_1d_141_2.npy', 'sequence_1d_141_2_label_.npy'), ('sequence_1k_230_1.npy', 'sequence_1k_230_1_label_.npy'), ('sequence_1l_27_8.npy', 'sequence_1l_27_8_label_.npy'), ('sequence_1a_31_2.npy', 'sequence_1a_31_2_label_.npy'), ('sequence_1b_81_2_fv1b.npy', 'sequence_1b_81_2_label_fv1b.npy'), ('sequence_2b_190_1_fv1b.npy', 'sequence_2b_190_1_label_fv1b.npy'), ('sequence_2b_197_8_fv1b.npy', 'sequence_2b_197_8_label_fv1b.npy'), ('sequence_1b_91_2_fv1b.npy', 'sequence_1b_91_2_label_fv1b.npy'), ('sequence_2b_215_6_fv1b.npy', 'sequence_2b_215_6_label_fv1b.npy'), ('sequence_1l_72_3.npy', 'sequence_1l_72_3_label_.npy'), ('sequence_1l_63_4.np

In [2]:
#features_list = ['percent_damage','delta_K_current_1','ctip_posn_curr_1','delta_K_current_2','ctip_posn_curr_2',
          #'delta_K_current_3','ctip_posn_curr_3','delta_K_current_4','ctip_posn_curr_4','Load_1','Load_2']
    
# train_list = ['StepIndex','delta_K_current_1','ctip_posn_curr_1','delta_K_current_2','ctip_posn_curr_2',
#           'delta_K_current_3','ctip_posn_curr_3','delta_K_current_4','ctip_posn_curr_4','Load_1','Load_2'] #and seq_id,somehow 
unique_init_conds = []
training_seqname_initcond_dict = {}
combined_seqname_initcond_dict = {}

for files in train_set_filenames:
    data_load_path = train_path + '/data/' + files[0]
    label_load_path = train_path + '/label/' + files[1]
    # print("data/label load path: {} \n {}".format(data_load_path,label_load_path))
    train_array = np.load(data_load_path)
    train_label_array = np.load(label_load_path)[:, 1:]
    if train_array.shape[1] == 12:
        train_array = train_array[:,1:] #skip step_index. 
    if "fv1b" not in str(files[0]):
        print(train_array[0,:])
        #init_cond = train_array[0,[3,5,7,9]]
        init_cond = train_array[0,[2,4,6,8]]
    if "fv1b" in str(files[0]):
        init_cond = train_array[0,[2,4,6,8]]
    #print(str(files[0])," init cond",init_cond)
    print(str(files[0])," init cond2",train_array[0,:])
    print(str(files[0])," init cond2",train_array[1,:])
    training_seqname_initcond_dict[str(files[0])] = str(init_cond)
    
    array_equality = False
    if len(unique_init_conds) == 0:
            unique_init_conds.append(init_cond)
    if len(unique_init_conds) != 0:
        for ic in unique_init_conds:
            if np.array_equal(init_cond,ic) == True:
                #print("equal array found in list?", np.array_equal(load_seq,load_sequence))
                array_equality = True
    if array_equality == False:
        unique_init_conds.append(init_cond) #the load types
    #print("first 6 rows:",train_array[0:2,:])
print("there are ",len(unique_init_conds)," init conds in the training set")
print("unique init conds:",unique_init_conds)

('sequence_2b_182_3_fv1b.npy', ' init cond2', array([ 19.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525 ,   0.    ,   2.2375,   1.428 ,  14.28  ]))
('sequence_2b_182_3_fv1b.npy', ' init cond2', array([ 19.00036379,   9.04254967,   0.72499818,   8.22923652,
         1.47499876,   8.52761875,   1.52500145,   8.1294277 ,
         2.2374986 ,  14.28      ,   1.25      ]))
('sequence_2b_222_3_fv1b.npy', ' init cond2', array([ 19.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525 ,   0.    ,   2.2375,   1.428 ,  12.28  ]))
('sequence_2b_222_3_fv1b.npy', ' init cond2', array([ 19.00016872,   7.15836975,   0.72499916,   7.15836975,
         1.47499916,   7.15836975,   1.52500084,   7.15836975,
         2.23749916,  12.28      ,   1.428     ]))
('sequence_1b_71_2_fv1b.npy', ' init cond2', array([  5.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525 ,   0.    ,   2.2375,   1.428 ,  14.28  ]))
('sequence_1b_71_2_fv1b.npy', ' init co

[ 27.       0.       0.3325   0.       1.2575   0.       1.4375   0.
   2.0525   1.428   14.28  ]
('sequence_1k_174_5.npy', ' init cond2', array([ 27.    ,   0.    ,   0.3325,   0.    ,   1.2575,   0.    ,
         1.4375,   0.    ,   2.0525,   1.428 ,  14.28  ]))
('sequence_1k_174_5.npy', ' init cond2', array([ 27.0003188 ,   8.95587129,   0.33249837,   8.23398686,
         1.25749872,   8.47363048,   1.43750134,   8.83350762,
         2.05249841,  14.28      ,   1.25      ]))
[ 27.       0.       0.3325   0.       1.2575   0.       1.4375   0.
   2.0525   1.428   14.28  ]
('sequence_1k_171_2.npy', ' init cond2', array([ 27.    ,   0.    ,   0.3325,   0.    ,   1.2575,   0.    ,
         1.4375,   0.    ,   2.0525,   1.428 ,  14.28  ]))
('sequence_1k_171_2.npy', ' init cond2', array([ 27.00032975,   7.8294729 ,   0.33249896,   7.84792116,
         1.25749893,   9.28012281,   1.43750195,   8.93889189,
         2.05249835,  14.28      ,   1.25      ]))
[  5.       0.       0.4125   0.  

('sequence_1c_40_1_fv1b.npy', ' init cond2', array([  5.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525 ,   0.    ,   2.2375,   1.428 ,  12.28  ]))
('sequence_1c_40_1_fv1b.npy', ' init cond2', array([  5.00016795,   6.98991749,   0.72499924,   7.24647916,
         1.47499916,   6.89505636,   1.52500074,   6.5354807 ,
         2.23749939,  12.28      ,   1.428     ]))
('sequence_2c_270_1_fv1b.npy', ' init cond2', array([  5.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525 ,   0.    ,   2.2375,   1.028 ,  14.28  ]))
('sequence_2c_270_1_fv1b.npy', ' init cond2', array([  5.00038196,   8.08397   ,   0.72499887,   9.51286894,
         1.47499809,   9.37595226,   1.52500186,   9.01847214,
         2.23749839,  14.28      ,   0.75      ]))
[ 27.       0.       0.3325   0.       1.2575   0.       1.4375   0.
   2.0525   1.428   14.28  ]
('sequence_1l_46_7.npy', ' init cond2', array([ 27.    ,   0.    ,   0.3325,   0.    ,   1.2575,   0.    ,
       

[  5.       0.       0.4125   0.       1.2875   0.       1.3375   0.
   2.175    1.428   12.28  ]
('sequence_1a_52_3.npy', ' init cond2', array([  5.    ,   0.    ,   0.4125,   0.    ,   1.2875,   0.    ,
         1.3375,   0.    ,   2.175 ,   1.428 ,  12.28  ]))
('sequence_1a_52_3.npy', ' init cond2', array([  5.00022681,   7.62269126,   0.41249887,   5.91982662,
         1.28749952,   7.66699655,   1.33750099,   7.1790794 ,
         2.17499915,  12.28      ,   1.428     ]))
[  5.       0.       0.4125   0.       1.2875   0.       1.3375   0.
   2.175    1.428   14.28  ]
('sequence_1a_26_7.npy', ' init cond2', array([  5.    ,   0.    ,   0.4125,   0.    ,   1.2875,   0.    ,
         1.3375,   0.    ,   2.175 ,   1.428 ,  14.28  ]))
('sequence_1a_26_7.npy', ' init cond2', array([  5.00029874,   8.47764173,   0.41249858,   8.47764173,
         1.28749851,   8.47764173,   1.33750132,   8.47764173,
         2.17499849,  14.28      ,   1.75      ]))
('sequence_2b_191_2_fv1b.npy', ' init 

[ 27.       0.       0.3325   0.       1.2575   0.       1.4375   0.
   2.0525   1.428   12.28  ]
('sequence_1l_50_1.npy', ' init cond2', array([ 27.    ,   0.    ,   0.3325,   0.    ,   1.2575,   0.    ,
         1.4375,   0.    ,   2.0525,   1.428 ,  12.28  ]))
('sequence_1l_50_1.npy', ' init cond2', array([ 27.00012554,   7.10593836,   0.33249918,   7.53005808,
         1.257499  ,   7.29109775,   1.4375009 ,   6.58220643,
         2.05249937,  12.28      ,   1.428     ]))
('sequence_1c_41_2_fv1b.npy', ' init cond2', array([  5.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525 ,   0.    ,   2.2375,   1.428 ,  12.28  ]))
('sequence_1c_41_2_fv1b.npy', ' init cond2', array([  5.00021877,   6.88213199,   0.72499932,   7.67044269,
         1.47499891,   7.67711757,   1.52500096,   7.25892792,
         2.23749907,  12.28      ,   1.428     ]))
('sequence_1c_20_1_fv1b.npy', ' init cond2', array([  5.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525

...

[ 27.       0.       0.3325   0.       1.2575   0.       1.4375   0.
   2.0525   1.428   14.28  ]
('sequence_1l_15_6.npy', ' init cond2', array([ 27.    ,   0.    ,   0.3325,   0.    ,   1.2575,   0.    ,
         1.4375,   0.    ,   2.0525,   1.428 ,  14.28  ]))
('sequence_1l_15_6.npy', ' init cond2', array([ 27.00032353,   7.98819911,   0.33249875,   8.48092743,
         1.25749856,   8.15950956,   1.43750125,   8.94812652,
         2.05249838,  14.28      ,   1.25      ]))
[ 27.       0.       0.3325   0.       1.2575   0.       1.4375   0.
   2.0525   1.428   14.28  ]
('sequence_1l_2_3.npy', ' init cond2', array([ 27.    ,   0.    ,   0.3325,   0.    ,   1.2575,   0.    ,
         1.4375,   0.    ,   2.0525,   1.428 ,  14.28  ]))
('sequence_1l_2_3.npy', ' init cond2', array([ 27.00034094,   8.62422299,   0.33249849,   8.90847159,
         1.25749832,   8.33734063,   1.43750135,   8.95785503,
         2.0524983 ,  14.28      ,   1.        ]))
[  5.       0.       0.4125   0.       1

[  5.       0.       0.4125   0.       1.2875   0.       1.3375   0.
   2.175    1.428   14.28  ]
('sequence_1a_10_1.npy', ' init cond2', array([  5.    ,   0.    ,   0.4125,   0.    ,   1.2875,   0.    ,
         1.3375,   0.    ,   2.175 ,   1.428 ,  14.28  ]))
('sequence_1a_10_1.npy', ' init cond2', array([  5.00032615,   8.37286691,   0.41249862,   8.85729859,
         1.28749837,   8.05178978,   1.3375012 ,   7.60028155,
         2.17499896,  14.28      ,   1.25      ]))
('sequence_2c_265_6_fv1b.npy', ' init cond2', array([  5.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525 ,   0.    ,   2.2375,   1.028 ,  10.28  ]))
('sequence_2c_265_6_fv1b.npy', ' init cond2', array([  5.00012152,   6.75012393,   0.7249994 ,   5.59105268,
         1.47499966,   6.71504324,   1.52500061,   5.6412997 ,
         2.23749965,  10.28      ,   0.75      ]))
('sequence_2b_224_5_fv1b.npy', ' init cond2', array([ 19.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.

('sequence_2b_170_1_fv1b.npy', ' init cond2', array([ 19.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525 ,   0.    ,   2.2375,   1.428 ,  14.28  ]))
('sequence_2b_170_1_fv1b.npy', ' init cond2', array([ 19.00025977,   8.33293208,   0.7249987 ,   8.26344128,
         1.47499881,   8.12045617,   1.52500135,   8.3095797 ,
         2.23749869,  14.28      ,   1.        ]))
('sequence_1c_4_5_fv1b.npy', ' init cond2', array([  5.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525 ,   0.    ,   2.2375,   1.428 ,  14.28  ]))
('sequence_1c_4_5_fv1b.npy', ' init cond2', array([  5.00029906,   8.42539124,   0.72499858,   8.60957823,
         1.4749985 ,   8.56258013,   1.52500148,   9.14970675,
         2.23749821,  14.28      ,   1.25      ]))
('sequence_1b_77_8_fv1b.npy', ' init cond2', array([  5.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525 ,   0.    ,   2.2375,   1.428 ,  14.28  ]))
('sequence_1b_77_8_fv1b.npy', ' init cond2'

('sequence_2b_214_5_fv1b.npy', ' init cond2', array([ 19.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525 ,   0.    ,   2.2375,   1.428 ,  14.28  ]))
('sequence_2b_214_5_fv1b.npy', ' init cond2', array([ 19.00027732,   8.47764173,   0.72499861,   8.47764173,
         1.47499867,   8.47764173,   1.52500136,   8.47764173,
         2.23749852,  14.28      ,   0.75      ]))
('sequence_2b_213_4_fv1b.npy', ' init cond2', array([ 19.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525 ,   0.    ,   2.2375,   1.428 ,  14.28  ]))
('sequence_2b_213_4_fv1b.npy', ' init cond2', array([ 19.00026265,   8.47764173,   0.72499869,   8.47764173,
         1.47499847,   8.47764173,   1.52500154,   8.47764173,
         2.23749859,  14.28      ,   0.75      ]))
('sequence_2b_195_6_fv1b.npy', ' init cond2', array([ 19.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525 ,   0.    ,   2.2375,   1.428 ,  14.28  ]))
('sequence_2b_195_6_fv1b.npy', ' init 

('sequence_2c_230_1_fv1b.npy', ' init cond2', array([  5.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525 ,   0.    ,   2.2375,   1.028 ,  11.28  ]))
('sequence_2c_230_1_fv1b.npy', ' init cond2', array([  5.00012261,   6.51191806,   0.72499945,   6.78033981,
         1.47499939,   6.12301059,   1.52500043,   7.10347209,
         2.23749924,  11.28      ,   1.        ]))
('there are ', 3, ' init conds in the training set')
('unique init conds:', [array([ 0.725 ,  1.475 ,  1.525 ,  2.2375]), array([ 0.3325,  1.2575,  1.4375,  2.0525]), array([ 0.4125,  1.2875,  1.3375,  2.175 ])])


In [3]:
unique_init_conds_test = []
test_seqname_initcond_dict = {}

for files in test_set_filenames:
    data_load_path = test_path + '/data/' + files[0]
    label_load_path = test_path + '/label/' + files[1]
    # print("data/label load path: {} \n {}".format(data_load_path,label_load_path))
    test_array = np.load(data_load_path)
    test_label_array = np.load(label_load_path)[:, 1:]
    if test_array.shape[1] == 12:
        test_array = test_array[:,1:] #skip step_index. 
    if "fv1b" not in str(files[0]):
        print(test_array[0,:])
        #init_cond = train_array[0,[3,5,7,9]]
        init_cond = test_array[0,[2,4,6,8]]
    if "fv1b" in str(files[0]):
        init_cond = test_array[0,[2,4,6,8]]
    print(str(files[0])," init cond",init_cond)
    test_seqname_initcond_dict[str(files[0])] = str(init_cond)
    
    array_equality = False
    if len(unique_init_conds_test) == 0:
            unique_init_conds_test.append(init_cond)
    if len(unique_init_conds_test) != 0:
        for ic in unique_init_conds_test:
            if np.array_equal(init_cond,ic) == True:
                #print("equal array found in list?", np.array_equal(load_seq,load_sequence))
                array_equality = True
    if array_equality == False:
        unique_init_conds_test.append(init_cond) #the load types
    #print("first 6 rows:",train_array[0:2,:])
print("there are ",len(unique_init_conds_test)," init conds in the TESTING set.", unique_init_conds_test)
print("and ", len(unique_init_conds_test), "in the TRAINING set.",unique_init_conds)
#print("unique init conds:",unique_init_conds_test)

#any(item.shape != (24,) for item in flattened_eq_len_loadseq_list)
#print("any non overlapping init conds?", cmp(unique_init_conds,unique_init_conds_test))

[ 27.       0.       0.3325   0.       1.2575   0.       1.4375   0.
   2.0525   1.428   14.28  ]
('sequence_1l_48_9.npy', ' init cond', array([ 0.3325,  1.2575,  1.4375,  2.0525]))
('sequence_2b_228_9_fv1b.npy', ' init cond', array([ 0.725 ,  1.475 ,  1.525 ,  2.2375]))
[ 27.       0.       0.3325   0.       1.2575   0.       1.4375   0.
   2.0525   1.428   14.28  ]
('sequence_1l_18_9.npy', ' init cond', array([ 0.3325,  1.2575,  1.4375,  2.0525]))
('sequence_2b_178_9_fv1b.npy', ' init cond', array([ 0.725 ,  1.475 ,  1.525 ,  2.2375]))
[  5.       0.       0.4125   0.       1.2875   0.       1.3375   0.
   2.175    1.428   14.28  ]
('sequence_1a_48_9.npy', ' init cond', array([ 0.4125,  1.2875,  1.3375,  2.175 ]))
('sequence_1b_79_10_fv1b.npy', ' init cond', array([ 0.725 ,  1.475 ,  1.525 ,  2.2375]))
('sequence_1b_88_9_fv1b.npy', ' init cond', array([ 0.725 ,  1.475 ,  1.525 ,  2.2375]))
('sequence_1c_19_10_fv1b.npy', ' init cond', array([ 0.725 ,  1.475 ,  1.525 ,  2.2375]))
('seq

[ 27.       0.       0.3325   0.       1.2575   0.       1.4375   0.
   2.0525   1.428   14.28  ]
('sequence_1l_39_10.npy', ' init cond', array([ 0.3325,  1.2575,  1.4375,  2.0525]))
[ 27.       0.       0.3325   0.       1.2575   0.       1.4375   0.
   2.0525   1.428   14.28  ]
('sequence_1d_98_9.npy', ' init cond', array([ 0.3325,  1.2575,  1.4375,  2.0525]))
[ 27.       0.       0.3325   0.       1.2575   0.       1.4375   0.
   2.0525   1.428   14.28  ]
('sequence_1d_109_10.npy', ' init cond', array([ 0.3325,  1.2575,  1.4375,  2.0525]))
('sequence_1b_58_9_fv1b.npy', ' init cond', array([ 0.725 ,  1.475 ,  1.525 ,  2.2375]))
[  5.       0.       0.4125   0.       1.2875   0.       1.3375   0.
   2.175    1.428   14.28  ]
('sequence_1a_29_10.npy', ' init cond', array([ 0.4125,  1.2875,  1.3375,  2.175 ]))
[  5.       0.       0.4125   0.       1.2875   0.       1.3375   0.
   2.175    1.428   14.28  ]
('sequence_1a_9_10.npy', ' init cond', array([ 0.4125,  1.2875,  1.3375,  2.175 ]

In [4]:
seqname_initcond_dict_initcondkey = {}

#combined_seqname_initcond_dict = {}

#do this twice. once for the training and once for the testing set
for key,value in training_seqname_initcond_dict.iteritems():
    print("key",key,"value",value)
    combined_seqname_initcond_dict[key[:-4]]=value
    
    #load_sequence_aslist = str(unique_load_sequences_aslists[value])
    #this arrangement below is necessary because dict has to have a list as its value. 
    if seqname_initcond_dict_initcondkey.get(value) != None:
        seqname_initcond_dict_initcondkey[value].append(key)
    if seqname_initcond_dict_initcondkey.get(value) == None: 
        content_list = []
        content_list.append(key[:-4])
        seqname_initcond_dict_initcondkey[value] = content_list

for key,value in test_seqname_initcond_dict.iteritems():
    print("key",key,"value",value)
    combined_seqname_initcond_dict[key[:-4]]=value
    #load_sequence_aslist = str(unique_load_sequences_aslists[value])
    #this arrangement below is necessary because dict has to have a list as its value. 
    if seqname_initcond_dict_initcondkey.get(value) != None:
        seqname_initcond_dict_initcondkey[value].append(key)
    if seqname_initcond_dict_initcondkey.get(value) == None: 
        content_list = []
        content_list.append(key[:-4])
        seqname_initcond_dict_initcondkey[value] = content_list

        
#print(filename_loadseq_dict_loadlistposkey)
sn_ic_ick_df = pd.DataFrame.from_dict(seqname_initcond_dict_initcondkey,orient='index')
sn_ic_ick_df.index.rename(name='init_cond',inplace=True)

sn_ic_snk_df = pd.DataFrame.from_dict(combined_seqname_initcond_dict,orient='index')
sn_ic_snk_df.index.rename(name='seq_name',inplace=True)
sn_ic_snk_df.columns=['init_cond']

sn_ic_ick_df.to_csv(analysis_path + "seqname_initcond_dict_initcondkey.csv") #the 3 init conds are index
sn_ic_snk_df.to_csv(analysis_path + "combined_seqname_initcond_dict.csv") #seq_name is index. to join later. 


('key', 'sequence_2c_232_3_fv1b.npy', 'value', '[ 0.725   1.475   1.525   2.2375]')
('key', 'sequence_1d_124_5.npy', 'value', '[ 0.3325  1.2575  1.4375  2.0525]')
('key', 'sequence_1l_27_8.npy', 'value', '[ 0.3325  1.2575  1.4375  2.0525]')
('key', 'sequence_1l_63_4.npy', 'value', '[ 0.3325  1.2575  1.4375  2.0525]')
('key', 'sequence_1b_97_8_fv1b.npy', 'value', '[ 0.725   1.475   1.525   2.2375]')
('key', 'sequence_1b_81_2_fv1b.npy', 'value', '[ 0.725   1.475   1.525   2.2375]')
('key', 'sequence_1a_20_1.npy', 'value', '[ 0.4125  1.2875  1.3375  2.175 ]')
('key', 'sequence_1l_42_3.npy', 'value', '[ 0.3325  1.2575  1.4375  2.0525]')
('key', 'sequence_1k_161_2.npy', 'value', '[ 0.3325  1.2575  1.4375  2.0525]')
('key', 'sequence_1a_53_4.npy', 'value', '[ 0.4125  1.2875  1.3375  2.175 ]')
('key', 'sequence_1a_75_6.npy', 'value', '[ 0.4125  1.2875  1.3375  2.175 ]')
('key', 'sequence_1a_17_8.npy', 'value', '[ 0.4125  1.2875  1.3375  2.175 ]')
('key', 'sequence_1l_15_6.npy', 'value', '[ 0.

In [5]:
#right df is the one described in the cell above. 
master_per_loadseq_df = pd.read_csv(analysis_path + 'master_score_per_loadseq.csv',index_col=0) #left. seq_name is the index. 
results_seqname_initcond_df = pd.merge(master_per_loadseq_df,sn_ic_snk_df,how='inner',left_index=True,right_index=True)
print(results_seqname_initcond_df.head(2))
print(results_seqname_initcond_df.columns)
print(results_seqname_initcond_df.index)


                    mse_avg   mse_med  mse_stdev   mae_avg   mae_med  \
seq_name                                                               
sequence_1d_99_10  0.066512  0.015284   0.108850  0.147706  0.069502   
sequence_1d_98_9   0.066754  0.015268   0.108441  0.147785  0.069892   

                   mae_stdev  mae_normd_dev       mape_avg      mape_med  \
seq_name                                                                   
sequence_1d_99_10   0.160956            NaN  578012.174698  11774.226492   
sequence_1d_98_9    0.160656            NaN  578054.191861  11778.379403   

                     mape_stdev  msle_avg  msle_med  msle_stdev  \
seq_name                                                          
sequence_1d_99_10  1.140165e+06       NaN       NaN         NaN   
sequence_1d_98_9   1.140265e+06       NaN       NaN         NaN   

                                           init_cond  
seq_name                                              
sequence_1d_99_10  [ 0.3325

In [6]:
columns_to_cast = list(results_seqname_initcond_df.columns)
columns_to_cast.remove('init_cond')
print(columns_to_cast)

for colname in columns_to_cast: #for some reason it's all objects...gotta cast to numeric. 
    results_seqname_initcond_df[str(colname)] = pd.to_numeric(results_seqname_initcond_df[str(colname)],errors='coerce')
for colname in columns_to_cast:
    print("new!", colname, results_seqname_initcond_df[colname].dtype)
grouped_sn_ic_df = results_seqname_initcond_df.groupby(by=['init_cond']).mean()

print(grouped_sn_ic_df.head(2))
#print(type(grouped_sn_ic_df))

pd.DataFrame(grouped_sn_ic_df).to_csv(analysis_path+'sn_ic_grouped_by_ic.csv')

['mse_avg', 'mse_med', 'mse_stdev', 'mae_avg', 'mae_med', 'mae_stdev', 'mae_normd_dev', 'mape_avg', 'mape_med', 'mape_stdev', 'msle_avg', 'msle_med', 'msle_stdev']
('new!', 'mse_avg', dtype('float64'))
('new!', 'mse_med', dtype('float64'))
('new!', 'mse_stdev', dtype('float64'))
('new!', 'mae_avg', dtype('float64'))
('new!', 'mae_med', dtype('float64'))
('new!', 'mae_stdev', dtype('float64'))
('new!', 'mae_normd_dev', dtype('float64'))
('new!', 'mape_avg', dtype('float64'))
('new!', 'mape_med', dtype('float64'))
('new!', 'mape_stdev', dtype('float64'))
('new!', 'msle_avg', dtype('float64'))
('new!', 'msle_med', dtype('float64'))
('new!', 'msle_stdev', dtype('float64'))
                                    mse_avg   mse_med  mse_stdev   mae_avg  \
init_cond                                                                    
[ 0.3325  1.2575  1.4375  2.0525]  0.095992  0.051050   0.118725  0.177471   
[ 0.4125  1.2875  1.3375  2.175 ]  0.133079  0.031265   0.175761  0.199008   

         

In [7]:
grouped_sn_ic_df2 = results_seqname_initcond_df.groupby(by=['init_cond']).count()
pd.DataFrame(grouped_sn_ic_df2).to_csv(analysis_path+'sn_ic_grouped_by_ic_count.csv')